In [3]:
# user-friendly print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
from joblib import Parallel, delayed

from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter, PercentFormatter
from collections import defaultdict
from crystallus import __version__, CrystalGenerator, WyckoffCfgGenerator
from crystallus.utils import WyckoffPositionConverter, build_structure, get_equivalent_coords 
from pymatgen.core import Structure
from pymatgen.analysis.structure_analyzer import SpacegroupAnalyzer

#### functions

In [5]:
from typing import List

def structure_dissimilarity(anchor_structure: Structure,
                            other_structures: List[Structure],
                            *,
                            verbose: int = 1,
                            n_jobs: int = 1):
    """Calculate dissimilarity between anchor and other structures.
    Parameters
    ----------
    anchor_structure:
        Anchor structure
    other_structures:
        Structures will be used to calculate the dissimilarity against the anchor structure.
    verbose:
        Verbose output when performing parallel calculation, by default 1
    n_jobs:
        Specify the number of cores for parallel calculation, by default 1
    Returns
    -------
    list
        List of dissimilarities.
    """
    # Calculate structure fingerprints.
    ssf = SiteStatsFingerprint(CrystalNNFingerprint.from_preset('ops',
                                                                distance_cutoffs=None,
                                                                x_diff_weight=0),
                               stats=('mean', 'std_dev', 'minimum', 'maximum'))
    v_anchor = np.array(ssf.featurize(anchor_structure))
    tmp = Parallel(n_jobs=n_jobs,
                   verbose=verbose)(delayed(ssf.featurize)(s) for s in other_structures)
    return [np.linalg.norm(np.array(s) - v_anchor) for s in tmp]

## Test case: `Ag32Ge4S24`.

* Spacegroup 33
* Wyckoff positions:
    (x,y,z) (-x,-y,z+1/2) (x+1/2,-y+1/2,z) (-x+1/2,y+1/2,z+1/2)
* Ground truth:
    {Ag: 4a \* 8, Ge: 4a, S: 4a \* 5}

In [7]:
ground_truth = Structure.from_file('cifs/mp-9770.cif')

print('abc: ', ground_truth.lattice.abc)
print('angles: ', ground_truth.lattice.angles)

abc:  (15.313245, 7.626161, 10.743155)
angles:  (90.0, 90.0, 90.0)


In [8]:
wg = WyckoffCfgGenerator(Ag=32, Ge=4, S=24)
wg

WyckoffCfgGenerator(            
    max_recurrent=1000,            
    n_jobs=-1            
    priority=None            
    composition={'Ag': 32, 'Ge': 4, 'S': 24}            
)

In [10]:
cfgs = wg.gen_many(1000, spacegroup_num=33)
cfgs

[{'Ag': ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'],
  'Ge': ['a'],
  'S': ['a', 'a', 'a', 'a', 'a', 'a']}]

Please note that space group 33 only has one Wyckoff position set.

In [11]:
cg = CrystalGenerator(
    spacegroup_num=33,
    volume_of_cell=1168.454590,
    variance_of_volume=15,
    verbose=False,
)
cg

CrystalGenerator(            
    spacegroup_num=33,            
    volume_of_cell=1168.45459,            
    variance_of_volume=15,            
    angle_range=(30.0, 150.0),            
    angle_tolerance=20.0,            
    max_attempts_number=5000,            
    lattice=None,            
    empirical_coords=None,            
    empirical_coords_variance=0.01,            
    empirical_coords_sampling_rate=1.0,            
    empirical_coords_loose_sampling=True,            
    verbose=False            
    n_jobs=-1            
)

## Test in most strict conditional

In [ ]:
%%time

expect_size = 100_000_000
results = []
ret = cg.gen_many(expect_size, cfgs[0], distance_scale_factor=0)
len(ret)

### Calculate the `distance_scale_factor` dependency

In [ ]:
%%time

expect_size = 50_000
results = []
radius = {"S": 1.02, "Ge": 1.18, "Ag": 1.4}
raw = np.array(
    [0.0, 0, 2.04, 2.20, 2.42, 2.36, 2.58, 2.8]
)

for d in np.linspace(0.2, 0.5, 16):
    ret = cg.gen_many(expect_size, cfgs[0], distance_scale_factor=d)
    ratio = len(ret) / expect_size
    raw_ = raw * (1 - d)
    raw_[0], raw_[1] = d, ratio
    results.append(raw_)

In [ ]:
results = pd.DataFrame(results, columns=['scale', 'proportion', 'S-S', 'S-Ge', 'S-Ag', 'Ge-Ge', 'Ge-Ag', 'Ag-Ag'])
results.head(3)

In [ ]:
f, ax_tolerance = plt.subplots(figsize=(10, 6), dpi=150)

results.plot.line(x='scale', y=['S-S', 'S-Ge', 'S-Ag', 'Ge-Ge', 'Ge-Ag', 'Ag-Ag'], linestyle='dashed', ax=ax_tolerance)
ax_tolerance.set_ylim(0, 2.5)
ax_tolerance.set_ylabel('Tolerance for atomic distance')
ax_tolerance.set_xlabel('Scale factor')
ax_tolerance.legend(loc='lower left', title='distance\ncondition')
ax_tolerance.grid(axis='y')
ax_tolerance.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax_size= ax_tolerance.twinx()
ax_size.bar(results.scale, results.proportion, 0.015)
ax_size.yaxis.set_major_formatter(PercentFormatter(1))
ax_size.set_ylabel('Proportion of successes')
ax_size.set_xlim(0.19, 0.51)

# f.savefig('Ag32Ge4S24_generation_analysis.png', bbox_inches='tight', dpi=300)

From the result of `distance_scale_factor` dependency. Set the `distance_scale_factor` to `0.45` seems to be a good choice.

Finally, let's generate some structures and calculate their dissimilarity with most stable structure.

In [ ]:
ground_truth = Structure.from_file('cifs/mp-9770.cif')

In [ ]:
%%time

expect_size = 1_000_000
cfg = {'Ag': ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'],
 'Ge': ['a'],
 'S': ['a', 'a', 'a', 'a', 'a', 'a']
}
results = []
ret = cg.gen_many(expect_size, cfg, distance_scale_factor=0.45)
len(ret)

In [ ]:
structures = Parallel(n_jobs=20, verbose=1)(delayed(build_structure)(s) for s in ret)
structures = pd.DataFrame(structures)
structures.to_pickle('fully_random_generated_structures.pd.xz')

In [ ]:
dissimilarity = structure_dissimilarity(ground_truth, structures.structure.tolist(), verbose=1, n_jobs=20)
structures = structures.assign(dissimilarity=dissimilarity)
structures.to_pickle('fully_random_generated_structures.pd.xz')

In [ ]:
f, ax = plt.subplots(dpi=200)
structures = pd.read_pickle('fully_random_generated_structures.pd.xz')
structures.dissimilarity.plot.hist(ax=ax, title='Fully random')

## Generation using empirical atomic coordinates

### loose pattern

In [ ]:
ag32ge4s24_like = pd.read_pickle('Ag32Ge4S24_like.pd.xz')
ag32ge4s24_like

There are five compounds has composition `(4:24:32)` in materials project database. We can use their coordinates as an empirical distribution to help our structure generation. 
First, let's load all these structures.

In [ ]:
ss = [Structure.from_file(f'cifs/{idx}.cif') for idx in ag32ge4s24_like.index[1:]]

Before going to the next step, I'd like to explain how to use exist coordinates as empirical coordinate distribution.
The most straight forward idea is assigning these coordinates to the generated structures by their Wyckoff position letters. Because Wyckoff position in the international table is written as a set of formulas which are something like this

```
(x,y,1/2)	(-y,x-y,1/2)	(-x+y,-x,1/2)	(-x,-y,1/2)
(y,-x+y,1/2)	(x-y,x,1/2)	(y,x,1/2)	(x-y,-y,1/2)
(-x,-x+y,1/2)	(-y,-x,1/2)	(-x+y,y,1/2)	(x,x-y,1/2)
```

So reuse a real coordinate can roughly be separated into the following steps.

1. calculate the equivalent coordinates for each site and get their Wyckoff position letter.
2. convert the fraction coordinates `(x, y, z)` into Wyckoff position coordinate `(x', y', z')`.
3. use converted coordinate to fulfill a structure.

We have prepared tool functions `get_equivalent_coords` and `WyckoffPositionConverter` for you to solve 1) and 2). For 3), this is exactly what the meaning of the parameter `empirical_coords` in `CrystalGenerator`.

In [ ]:
get_equivalent_coords?

In [ ]:
equivalent_coords = get_equivalent_coords(ss[0])
equivalent_coords

convert to Wyckoff position format

In [ ]:
spg_num = equivalent_coords.spacegroup_num[0]
converter = WyckoffPositionConverter(spg_num)
converter?

In [ ]:
xyzs = converter('wyckoff_letter', 'coordinate', data=equivalent_coords)
xyzs

Apply 1) and 2) for all these five structures. 

In [ ]:
equivalent_coords =  pd.concat([get_equivalent_coords(s) for s in ss])
xyzs = converter('wyckoff_letter', 'coordinate', data=equivalent_coords)
len(xyzs)

Give these coordinates to the generator, and set `empirical_coords_variance` to give each coordinate a perturbation.
`empirical_coords_variance` has default value 0.01, and will be used to build normal distribution $N(0, \textrm{empirical_coords_variance})$ 

In [ ]:
cg = CrystalGenerator(
    spacegroup_num=spg_num,
    volume_of_cell=1254.599172,
    variance_of_volume=10,
    empirical_coords=xyzs,
    empirical_coords_variance=0.02,
    verbose=False
)
cg

generation

In [ ]:
%%time

expect_size = 1_000_000
cfg = {'Ag': ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'],
 'Ge': ['a'],
 'S': ['a', 'a', 'a', 'a', 'a', 'a']
}
results = []
ret = cg.gen_many(expect_size, cfg, distance_scale_factor=0.45)
len(ret)

In [ ]:
%%time

expect_size = 50_000
results = []
radius = {"S": 1.02, "Ge": 1.18, "Ag": 1.4}
raw = np.array(
    [0.0, 0, 2.04, 2.20, 2.42, 2.36, 2.58, 2.8]
)

for d in np.linspace(0.2, 0.5, 16):
    ret = cg.gen_many(expect_size, cfgs[0], distance_scale_factor=d)
    ratio = len(ret) / expect_size
    raw_ = raw * (1 - d)
    raw_[0], raw_[1] = d, ratio
    results.append(raw_)

In [ ]:
results = pd.DataFrame(results, columns=['scale', 'proportion', 'S-S', 'S-Ge', 'S-Ag', 'Ge-Ge', 'Ge-Ag', 'Ag-Ag'])
f, ax_tolerance = plt.subplots(figsize=(10, 6), dpi=150)

results.plot.line(x='scale', y=['S-S', 'S-Ge', 'S-Ag', 'Ge-Ge', 'Ge-Ag', 'Ag-Ag'], linestyle='dashed', ax=ax_tolerance)
ax_tolerance.set_ylim(0, 2.5)
ax_tolerance.set_ylabel('Tolerance for atomic distance')
ax_tolerance.set_xlabel('Scale factor')
ax_tolerance.legend(loc='lower left', title='distance\ncondition')
ax_tolerance.grid(axis='y')
ax_tolerance.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax_size= ax_tolerance.twinx()
ax_size.bar(results.scale, results.proportion, 0.015)
ax_size.yaxis.set_major_formatter(PercentFormatter(1))
ax_size.set_ylabel('Proportion of successes')
ax_size.set_xlim(0.19, 0.51)

# f.savefig('Ag32Ge4S24_generation_analysis.png', bbox_inches='tight', dpi=300)

In [ ]:
%%time

expect_size = 1_000_000
cfg = {'Ag': ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'],
 'Ge': ['a'],
 'S': ['a', 'a', 'a', 'a', 'a', 'a']
}
results = []
ret = cg.gen_many(expect_size, cfg, distance_scale_factor=0.45)
len(ret)

In [ ]:
structures = Parallel(n_jobs=20, verbose=1)(delayed(build_structure)(s) for s in ret)
structures = pd.DataFrame(structures)
structures.to_pickle('template_based_generated_structures.pd.xz')

In [ ]:
dissimilarity = structure_dissimilarity(ground_truth, structures.structure.tolist(), verbose=1, n_jobs=20)
structures = structures.assign(dissimilarity=dissimilarity)
structures.to_pickle('template_based_generated_structures.pd.xz')

In [ ]:
f, ax = plt.subplots(dpi=200)
structures = pd.read_pickle('template_based_generated_structures.pd.xz')
structures.dissimilarity.plot.hist(ax=ax, title='Loose matching')

### in strict pattern

In [ ]:
ag32ge4s24_like

In [ ]:
def mapper(elem, wy, mul):
    if elem in ['Ag', 'Li', 'Na']:
        return 'Ag'
    if elem in ['Ti', 'Si']:
        return 'Ge'
    return 'S'

In [ ]:
equivalent_coords =  pd.concat([get_equivalent_coords(s, mapper=mapper) for s in ss])
xyzs = converter('wyckoff_letter', 'coordinate', 'target_element', data=equivalent_coords)
len(xyzs)

In [ ]:
cg = CrystalGenerator(
    spacegroup_num=spg_num,
    volume_of_cell=1254.599172,
    variance_of_volume=10,
    empirical_coords=xyzs,
    empirical_coords_variance=0.02,
    empirical_coords_loose_sampling=False,
    verbose=False
)
cg

generation

In [ ]:
%%time

expect_size = 50_000
results = []
radius = {"S": 1.02, "Ge": 1.18, "Ag": 1.4}
raw = np.array(
    [0.0, 0, 2.04, 2.20, 2.42, 2.36, 2.58, 2.8]
)

for d in np.linspace(0.2, 0.5, 16):
    ret = cg.gen_many(expect_size, cfgs[0], distance_scale_factor=d)
    ratio = len(ret) / expect_size
    raw_ = raw * (1 - d)
    raw_[0], raw_[1] = d, ratio
    results.append(raw_)

In [ ]:
results = pd.DataFrame(results, columns=['scale', 'proportion', 'S-S', 'S-Ge', 'S-Ag', 'Ge-Ge', 'Ge-Ag', 'Ag-Ag'])
f, ax_tolerance = plt.subplots(figsize=(10, 6), dpi=150)

results.plot.line(x='scale', y=['S-S', 'S-Ge', 'S-Ag', 'Ge-Ge', 'Ge-Ag', 'Ag-Ag'], linestyle='dashed', ax=ax_tolerance)
ax_tolerance.set_ylim(0, 2.5)
ax_tolerance.set_ylabel('Tolerance for atomic distance')
ax_tolerance.set_xlabel('Scale factor')
ax_tolerance.legend(loc='lower left', title='distance\ncondition')
ax_tolerance.grid(axis='y')
ax_tolerance.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax_size= ax_tolerance.twinx()
ax_size.bar(results.scale, results.proportion, 0.015)
ax_size.yaxis.set_major_formatter(PercentFormatter(1))
ax_size.set_ylabel('Proportion of successes')
ax_size.set_xlim(0.19, 0.51)

# f.savefig('Ag32Ge4S24_generation_analysis.png', bbox_inches='tight', dpi=300)

In [ ]:
%%time

expect_size = 1_000_000
cfg = {'Ag': ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'],
 'Ge': ['a'],
 'S': ['a', 'a', 'a', 'a', 'a', 'a']
}
results = []
ret = cg.gen_many(expect_size, cfg, distance_scale_factor=0.45)
len(ret)

In [ ]:
structures = Parallel(n_jobs=20, verbose=1)(delayed(build_structure)(s) for s in ret)
structures = pd.DataFrame(structures)
structures.to_pickle('strict_generated_structures.pd.xz')

In [ ]:
dissimilarity = structure_dissimilarity(ground_truth, structures.structure.tolist(), verbose=1, n_jobs=20)
structures = structures.assign(dissimilarity=dissimilarity)
structures.to_pickle('strict_generated_structures.pd.xz')

In [ ]:
f, ax = plt.subplots(dpi=200)
structures = pd.read_pickle('strict_generated_structures.pd.xz')
structures.dissimilarity.plot.hist(ax=ax, title='Strict matching')

### sampling from groundtruth coords

In [ ]:
equivalent_coords = get_equivalent_coords(ground_truth)

In [ ]:
spg_num = equivalent_coords.spacegroup_num[0]
converter = WyckoffPositionConverter(spg_num)
xyzs = converter('wyckoff_letter', 'coordinate', data=equivalent_coords)
len(xyzs)

In [ ]:
# loose pattern

cg = CrystalGenerator(
    spacegroup_num=spg_num,
    volume_of_cell=1254.599172,
    variance_of_volume=10,
    empirical_coords=xyzs,
    empirical_coords_variance=0.,
    empirical_coords_loose_sampling=True,
    verbose=False
)
cg

In [ ]:
%%time

expect_size = 50_000
results = []
radius = {"S": 1.02, "Ge": 1.18, "Ag": 1.4}
raw = np.array(
    [0.0, 0, 2.04, 2.20, 2.42, 2.36, 2.58, 2.8]
)

for d in np.linspace(0.2, 0.5, 16):
    ret = cg.gen_many(expect_size, cfgs[0], distance_scale_factor=d)
    ratio = len(ret) / expect_size
    raw_ = raw * (1 - d)
    raw_[0], raw_[1] = d, ratio
    results.append(raw_)

In [ ]:
results = pd.DataFrame(results, columns=['scale', 'proportion', 'S-S', 'S-Ge', 'S-Ag', 'Ge-Ge', 'Ge-Ag', 'Ag-Ag'])
f, ax_tolerance = plt.subplots(figsize=(10, 6), dpi=150)

# results.plot.line(x='scale', y=['S-S', 'S-Ge', 'S-Ag', 'Ge-Ge', 'Ge-Ag', 'Ag-Ag'], linestyle='dashed', ax=ax_tolerance)
# ax_tolerance.set_ylim(0, 2.5)
# ax_tolerance.set_ylabel('Tolerance for atomic distance')
# ax_tolerance.set_xlabel('Scale factor')
# ax_tolerance.legend(loc='lower left', title='distance\ncondition')
# ax_tolerance.grid(axis='y')
# ax_tolerance.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))

# ax_size= ax_tolerance.twinx()
ax_size= ax_tolerance
ax_size.bar(results.scale, results.proportion, 0.015)
ax_size.yaxis.set_major_formatter(PercentFormatter(1))
ax_size.set_ylabel('Proportion of successes')
ax_size.set_xlim(0.19, 0.51)

# f.savefig('Ag32Ge4S24_generation_analysis.png', bbox_inches='tight', dpi=300)

In [ ]:
%%time

expect_size = 10_000
cfg = {'Ag': ['a'] * 8,
 'Ge': ['a'],
 'S': ['a'] * 6
}
results = []
ret = cg.gen_many(expect_size, cfg, distance_scale_factor=0.45)
len(ret)

In [ ]:
structures = Parallel(n_jobs=20, verbose=1)(delayed(build_structure)(s) for s in ret)
structures = pd.DataFrame(structures)
structures.to_pickle('real_coord_structures.pd.xz')

In [ ]:
for idx, struct in structures.structure.items():
    true_coord = set([str(s) for s in ground_truth.frac_coords])
    if not set([str(s) for s in struct.frac_coords]) == true_coord:
        print(idx)

In [ ]:
dissimilarity = structure_dissimilarity(ground_truth, structures.structure.tolist(), verbose=1, n_jobs=20)
structures = structures.assign(dissimilarity=dissimilarity)
structures.to_pickle('real_coord_structures.pd.xz')

In [ ]:
f, ax = plt.subplots(dpi=200)
structures = pd.read_pickle('real_coord_structures.pd.xz')
structures.dissimilarity.plot.hist(ax=ax, title='Groundtruth coords')

generation

In [ ]:
dissimilarity = structure_dissimilarity(ground_truth, ss, verbose=1, n_jobs=1)

In [ ]:
f, ax = plt.subplots(dpi=200)
# structures = pd.read_pickle('real_coord_structures.pd.xz')
structures.dissimilarity.plot.hist(ax=ax, title='Random Lattice only')